In [1]:
import pandas as pd
import numpy as np

In [14]:
close_1 = pd.read_csv('close_1.csv')
close_2 = pd.read_csv('close_2.csv')
open_1 = pd.read_csv('open_1.csv')
open_2 = pd.read_csv('open_2.csv')

close_data = pd.merge(close_1, close_2)
open_data = pd.merge(open_1, open_2)
close_data = close_data.set_index(['Date'])
open_data = open_data.set_index(['Date'])
close_data = close_data.reindex(sorted(close_data.columns), axis=1)
open_data = open_data.reindex(sorted(open_data.columns), axis=1)

In [62]:
tickers = ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'PFE']
hist_data = []

for ticker in tickers:
    hist_data.append(get_data(ticker, start_date = '1/1/2012', 
                              end_date = '12/31/2022', index_as_date = True))

open_price = []
close_price = []
for i in range(len(hist_data)):
    open_price.append(hist_data[i]['open'])
    close_price.append(hist_data[i]['adjclose'])
    
open_data = pd.DataFrame(open_price, tickers)
open_data = open_data.transpose()
open_data = open_data.reindex(sorted(open_data.columns), axis=1)

close_data = pd.DataFrame(close_price, tickers)
close_data = close_data.transpose()
close_data = close_data.reindex(sorted(close_data.columns), axis=1)

In [22]:
daytime_return = []
daily_return = pd.DataFrame()
overnight_return = []

tickers = close_data.columns

for ticker in open_data:
    daytime_return.append((close_data[ticker] / open_data[ticker]) - 1)
    
    daily = []
    for j in range(1, len(close_data[ticker])):
        daily.append((1 - (close_data[ticker][j-1]  / close_data[ticker][j])) * 100)
    daily_return[ticker] = daily

daytime_return = pd.DataFrame(daytime_return, tickers)
daytime_return = daytime_return.transpose()

daily_return['Date'] = (close_data.index.tolist())[1:]
daily_return = daily_return.set_index('Date')

for ticker in open_data:
    overnight_return.append(((1 + daily_return[ticker]) / (1 + daytime_return[ticker])) - 1)
    
overnight_return = pd.DataFrame(overnight_return, tickers)
overnight_return = overnight_return.transpose()

reversals = pd.DataFrame()

for ticker in overnight_return:
    rev = []
    for j in range(len(overnight_return[ticker])):
        if (overnight_return[ticker][j] > 0 and daytime_return[ticker][j] < 0):
            rev.append("Negative")
        elif (overnight_return[ticker][j] < 0 and daytime_return[ticker][j] > 0):
            rev.append("Positive")
        else:
            rev.append(False)
    reversals[ticker] = rev

reversals['Date'] = pd.to_datetime(daytime_return.index.tolist())
reversals['year_month'] = reversals['Date'].dt.to_period('M')
reversals['lag'] = reversals['year_month'].shift(1)

pos_ratio = pd.DataFrame()
neg_ratio = pd.DataFrame()

for ticker in reversals:
    
    pos_rat = []
    neg_rat = []
    
    neg_count = 0
    pos_count = 0
    tot = 0
    for j in range(len(reversals[ticker])):
        if (reversals['year_month'][j]) == (reversals['lag'][j]) or type(reversals['lag'][j]) == pd._libs.tslibs.nattype.NaTType:
            if reversals[ticker][j] == 'Negative':
                neg_count += 1
            elif reversals[ticker][j] == 'Positive':
                pos_count += 1
            tot += 1
        else:
            pos_rat.append(pos_count / tot)
            neg_rat.append(neg_count / tot)
            neg_count = 0
            pos_count = 0
            tot = 0 
        if j == (len(reversals[ticker])-1):
            pos_rat.append(pos_count / tot)
            neg_rat.append(neg_count / tot)
    pos_ratio[ticker] = pos_rat
    neg_ratio[ticker] = neg_rat
    
pos_ratio['Date'] = reversals['year_month'].unique()
neg_ratio['Date'] = reversals['year_month'].unique()

pos_ratio = pos_ratio.set_index('Date')
neg_ratio = neg_ratio.set_index('Date')
reversals = reversals.set_index('Date')

pos_ratio = pos_ratio.drop(['year_month', 'lag'], axis = 1)
neg_ratio = neg_ratio.drop(['year_month', 'lag'], axis = 1)
reversals = reversals.drop(['year_month', 'lag'], axis = 1)

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  daily_return[ticker] = daily
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

...

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pos_ratio[ticker] = pos_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  neg_ratio[ticker] = neg_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pos_ratio[ticker] = pos_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  neg_ratio[ticker] = neg_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pos_ratio[ticker] = pos_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  neg_ratio[ticker] = neg_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pos_ratio[ticker] = pos_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  neg_ratio[ticker] = neg_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pos_ratio[ticker] = pos_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:74: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  neg_ratio[ticker] = neg_rat
C:\Users\hanna\AppData\Local\Temp\ipykernel_28416\690802876.py:73: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [23]:
neg_windows = neg_ratio.rolling(12) 
neg_rolling_avg = neg_windows.mean()[12:]

pos_windows = pos_ratio.rolling(12) 
pos_rolling_avg = pos_windows.mean()[12:]

ab_pos_ratio = (pos_ratio * pos_rolling_avg)[12:]
ab_neg_ratio = (neg_ratio * neg_rolling_avg)[12:]

In [50]:
daytime_return = daytime_return.loc[(daytime_return.index >= '2013-01-01')]
daily_return = daily_return.loc[(daily_return.index >= '2013-01-01')]
overnight_return = overnight_return.loc[(overnight_return.index >= '2013-01-01')]
pos_ratio = pos_ratio.loc[(pos_ratio.index >= '2013-01')]
neg_ratio = neg_ratio.loc[(neg_ratio.index >= '2013-01')]

In [51]:
daytime_return.to_csv('daytime_returns.csv')
daily_return.to_csv('daily_returns.csv')
overnight_return.to_csv('overnight_returns.csv')
pos_ratio[12:].to_csv('positive_ratio.csv')
neg_ratio[12:].to_csv('negative_ratio.csv')
ab_pos_ratio.to_csv('abnormal_positive_ratio.csv')
ab_neg_ratio.to_csv('abnormal_negative_ratio.csv')